In [ ]:
import tweepy
import json
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import seaborn as sns

In [ ]:
# all 4 authentication keys to access twitter API
# to connect as OAth handler or jump serever / twitters  proxy server
consumer_key = ""
consumer_sec = ""


# from proxy server we need to connect
access_token = ""
access_token_sec = ""

In [ ]:
# connect to jump server of twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_sec)

In [ ]:
# now we can connect from jump server to web server of twitter
auth.set_access_token(access_token,access_token_sec)

In [ ]:
#connect to API storage server of twitter
api_connect = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
keyword = input("Enter your word: ")
new_search = keyword + " -filter:retweets"
number = 100
#tweets=api_connect.search(keyword,count=number, lang='en')

tweets = tweepy.Cursor(api_connect.search_tweets, new_search, lang='en').items(number)

#abc=[tweet.text for tweet in tweets]
abc = [[tweet.user.screen_name, tweet.text] for tweet in tweets]

Enter your word: russia


In [ ]:
tweet_text = pd.DataFrame(data=abc, 
                    columns=['Name', "Text"])

In [ ]:
tweet_text

,Name,Text
0,stwickery,@alpharith @Drezdenwiz @ggreenwald For over a ...
1,SocratesPoly,@IWashington #1 - China can have Taiwan...prob...
2,gahamalian,WITH OFFICIALS LIKE THESE… Mr. \n\nRussia need...
3,no2wipp,@markX420 the big russia-china show is just an...
4,Robert_Patman,The environmental impact of Russia’s invasion ...
...,...,...
95,hobeanyo,"Norway, Sweden, Finland, and Denmark struck a ..."
96,AutonomousA2,@PROUDKYAMERICAN @intertubenube He would end a...
97,MichaelSDTX,The Consecration of Russia: One Year Later - O...
98,p1ner0,$$$\nThis market is going to be different in a...


In [ ]:
tweet_text['Text'][0]

'@alpharith @Drezdenwiz @ggreenwald For over a decade every leading newspaper in every western democracy wrote about… https://t.co/QgV9rZNFxd'

## remove all the words starting with @


In [ ]:
# define a lambda function to remove words starting with "@"
remove_at = lambda x: re.sub(r'@\w+', '', x)

# apply the lambda function to each cell in column text in the dataframe
tweet_text["Text"] = tweet_text["Text"].apply(remove_at)


##removing links from text

In [ ]:
pattern = r'http\S+'

# replace links with an empty string in the text column
tweet_text['Text'] = tweet_text['Text'].replace(to_replace=pattern, value='', regex=True)

## Removing symbols form text

In [ ]:
# define regular expression pattern to match symbols that are not word or white space 
pattern = r'[^\w\s]'

# replace symbols with an empty string in the text column
tweet_text['Text'] = tweet_text['Text'].replace(to_replace=pattern, value='', regex=True)

In [ ]:
tweet_text

,Name,Text
0,stwickery,For over a decade every leading newspaper i...
1,SocratesPoly,1 China can have Taiwanprob will\n2 China h...
2,gahamalian,WITH OFFICIALS LIKE THESE Mr \n\nRussia needs ...
3,no2wipp,the big russiachina show is just anothe bread...
4,Robert_Patman,The environmental impact of Russias invasion g...
...,...,...
95,hobeanyo,Norway Sweden Finland and Denmark struck a dea...
96,AutonomousA2,He would end a war between Russia and Ukrain...
97,MichaelSDTX,The Consecration of Russia One Year Later One...
98,p1ner0,\nThis market is going to be different in a co...


In [ ]:
tweet_text[['Polarity', 'Subjectivity']] = tweet_text['Text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
tweet_text

,Name,Text,Polarity,Subjectivity
0,stwickery,For over a decade every leading newspaper i...,0.000000,0.000000
1,SocratesPoly,1 China can have Taiwanprob will\n2 China h...,0.317857,0.542857
2,gahamalian,WITH OFFICIALS LIKE THESE Mr \n\nRussia needs ...,0.000000,0.000000
3,no2wipp,the big russiachina show is just anothe bread...,0.000000,0.100000
4,Robert_Patman,The environmental impact of Russias invasion g...,0.000000,0.000000
...,...,...,...,...
95,hobeanyo,Norway Sweden Finland and Denmark struck a dea...,0.164286,0.407143
96,AutonomousA2,He would end a war between Russia and Ukrain...,0.000000,0.000000
97,MichaelSDTX,The Consecration of Russia One Year Later One...,0.000000,0.000000
98,p1ner0,\nThis market is going to be different in a co...,0.000000,0.600000


In [ ]:
# define a lambda function to replace numbers with "positive" or "negative"
replace_num = lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral'
# apply the lambda function to the column containing numbers
tweet_text['Polarity'] = tweet_text['Polarity'].apply(replace_num)

In [ ]:
tweet_text

,Name,Text,Polarity,Subjectivity
0,stwickery,For over a decade every leading newspaper i...,Neutral,0.000000
1,SocratesPoly,1 China can have Taiwanprob will\n2 China h...,Positive,0.542857
2,gahamalian,WITH OFFICIALS LIKE THESE Mr \n\nRussia needs ...,Neutral,0.000000
3,no2wipp,the big russiachina show is just anothe bread...,Neutral,0.100000
4,Robert_Patman,The environmental impact of Russias invasion g...,Neutral,0.000000
...,...,...,...,...
95,hobeanyo,Norway Sweden Finland and Denmark struck a dea...,Positive,0.407143
96,AutonomousA2,He would end a war between Russia and Ukrain...,Neutral,0.000000
97,MichaelSDTX,The Consecration of Russia One Year Later One...,Neutral,0.000000
98,p1ner0,\nThis market is going to be different in a co...,Neutral,0.600000


In [ ]:
tweet_text.dropna(inplace=True)

In [ ]:
tweet_text.to_csv('sentiment.csv',index=False)